
# Introduction to Machine Learning

An interactive Machine Learning example written in Python, broken into the following sections:

1. [Scope](#scope)
  1. [Problem Definition](#scope-problem-definition)
  1. [Data](#scope-data)
1. [Prepare](#prepare)
  1. [Import](#prepare-import)
  1. [Cleaning](#prepare-cleaning)
1. [Analyze](#analyze)
  1. [Pre-processing](#analyze-pre-processing)
  1. [Modeling](#analyze-modeling)
  1. [Hyper-parameter Tuning](#analyze-hyper-parameter-tuning)
1. [Reflect](#reflect)

<a id='scope'></a>

## Scope

<a id='scope-problem-definition'></a>

### Problem Definition

We seek to answer the following questions by applying Machine Learning techniques to the NYC Mayor's Office of Sustainability dataset on green buildings and energy efficiency:

1. Given a buidling's characteristics, can we predict what its energy usage intensity will be?
  - Could be useful for urban planners to proactively estimate energy consumption of various of living spaces.
1. Given a buidling's characteristics and energy use, can we predict what its Energy Star rating would be?
  - Could be useful for Energy Star to assist in assigning ratings to buildings without a score.
  - Could be useful for architects to predict the Energy Star scores of different designs.
1. What sources of energy cause the most greenhouse gas emissions for a property?
  - Could be useful for policy makers to see what sources of energy produce the most greenhouse gas emissions.
1. What factors lead to energy loss between the source of generation and the site of the property?
  - Could be useful for engineers to identify which locations, property usages, sources of energy, etc. lead to the most energy loss and adjust the energy grid accordingly.

<a id='scope-data'></a>

### Data

#### Citation

```
NYC Mayor's Office of Sustainability, Green Buildings & Energy Efficiency. (2017). [Data set]. 
    Retrieved from http://www.nyc.gov/html/gbee/html/plan/ll84_scores.shtml
```

#### Dimensions

| Column                                                     | Functional Data Type    | Technical Data Type |
|------------------------------------------------------------|-------------------------|---------------------|
| Order                                                      | index                   | integer             |
| Property Id                                                | identifier              | text                |
| Property Name                                              | identifier              | text                |
| Parent Property Id                                         | identifier              | text                |
| Parent Property Name                                       | identifier              | text                |
| BBL - 10 digits                                            | identifier              | text                |
| NYC Borough, Block and Lot (BBL) self-reported             | identifier              | text                |
| NYC Building Identification Number (BIN)                   | identifier              | text                |
| Address 1 (self-reported)                                  | location                | text                |
| Address 2                                                  | location                | text                |
| Postal Code                                                | location                | text                |
| Street Number                                              | location                | text                |
| Street Name                                                | location                | text                |
| Borough                                                    | location                | catagorical         |
| DOF Gross Floor Area                                       | building characteristic | numeric             |
| Primary Property Type - Self Selected                      | building characteristic | categorical         |
| List of All Property Use Types at Property                 | building characteristic | [categorical]       |
| Largest Property Use Type                                  | building characteristic | categorical         |
| Largest Property Use Type - Gross Floor Area (ft²)         | building characteristic | numeric             |
| 2nd Largest Property Use Type                              | building characteristic | categorical         |
| 2nd Largest Property Use - Gross Floor Area (ft²)          | building characteristic | numeric             |
| 3rd Largest Property Use Type                              | building characteristic | categorical         |
| 3rd Largest Property Use Type - Gross Floor Area (ft²)     | building characteristic | numeric             |
| Year Built                                                 | building characteristic | time                |
| Number of Buildings - Self-reported                        | building characteristic | integer             |
| Occupancy                                                  | building characteristic | percentage          |
| Metered Areas (Energy)                                     | building characteristic | categorical         |
| Metered Areas (Water)                                      | building characteristic | categorical         |
| ENERGY STAR Score                                          | score                   | numeric             |
| Site EUI (kBtu/ft²)                                        | energy usage            | numeric             |
| Weather Normalized Site EUI (kBtu/ft²)                     | energy usage            | numeric             |
| Weather Normalized Site Electricity Intensity (kWh/ft²)    | energy usage            | numeric             |
| Weather Normalized Site Natural Gas Intensity (therms/ft²) | energy usage            | numeric             |
| Weather Normalized Source EUI (kBtu/ft²)                   | energy usage            | numeric             |
| Fuel Oil #1 Use (kBtu)                                     | energy usage            | numeric             |
| Fuel Oil #2 Use (kBtu)                                     | energy usage            | numeric             |
| Fuel Oil #4 Use (kBtu)                                     | energy usage            | numeric             |
| Fuel Oil #5 &amp; 6 Use (kBtu)                             | energy usage            | numeric             |
| Diesel #2 Use (kBtu)                                       | energy usage            | numeric             |
| District Steam Use (kBtu)                                  | energy usage            | numeric             |
| Natural Gas Use (kBtu)                                     | energy usage            | numeric             |
| Weather Normalized Site Natural Gas Use (therms)           | energy usage            | numeric             |
| Electricity Use - Grid Purchase (kBtu)                     | energy usage            | numeric             |
| Weather Normalized Site Electricity (kWh)                  | energy usage            | numeric             |
| Total GHG Emissions (Metric Tons CO2e)                     | environmental footprint | numeric             |
| Direct GHG Emissions (Metric Tons CO2e)                    | environmental footprint | numeric             |
| Indirect GHG Emissions (Metric Tons CO2e)                  | environmental footprint | numeric             |
| Property GFA - Self-Reported (ft²)                         | building characteristic | numeric             |
| Water Use (All Water Sources) (kgal)                       | resource usage          | numeric             |
| Water Intensity (All Water Sources) (gal/ft²)              | resource usage          | numeric             |
| Source EUI (kBtu/ft²)                                      | energy usage            | numeric             |
| Release Date                                               | metadata                | time                |
| Water Required?                                            | building characteristic | categorical         |
| DOF Benchmarking Submission Status                         | metadata                | categorical         |

<a id='prepare'></a>

## Prepare

<a id='prepare-import'></a>

### Import

In [1]:
import os
import pprint
import numpy as np
import pandas as pd

In [ ]:
# Import the data
data_path = os.path.join(os.path.abspath('..'), 'data', 'nyc_benchmarking_disclosure_data_reported_in_2017.xlsx')
df = pd.read_excel(data_path, sheet_name='Information and Metrics')

In [ ]:
# View the structure of the data
df.head()

<a id='prepare-cleaning'></a>

### Cleaning

#### Missing Values

In [ ]:
# Note that NaNs are listed as "Not Available" in this data set
# Change these to NaN
df = df.replace('Not Available', np.NaN)

# Display the percentage of missing values for each column
pd.options.display.float_format = '{:.2f}'.format
percent_nan = df.isna().sum()/len(df) * 100
print(percent_nan)

In [ ]:
# Remove all columns that have more than 33% missing values
df = df.loc[:, percent_nan < 33]

#### Column Names

In [ ]:
# Look at the existing column names
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(list(df.columns))

In [ ]:
# Rename columns so that they are easier to reference
df = df.rename(index=str, columns={"Order": "order", 
                              "Property Id": "id", 
                              "Property Name": "name", 
                              "Parent Property Id": "prnt_id", 
                              "Parent Property Name": "prnt_name", 
                              "BBL - 10 digits": "brgh_blck_lt", 
                              "NYC Borough, Block and Lot (BBL) self-reported": "brgh_blck_lt_self", 
                              "NYC Building Identification Number (BIN)": "bldng_id_no", 
                              "Address 1 (self-reported)": "addr_1_self", 
                              "Postal Code": "zip_code", 
                              "Street Number": "st_no", 
                              "Street Name": "st_name", 
                              "Borough": "brgh", 
                              "DOF Gross Floor Area": "flr_area", 
                              "Primary Property Type - Self Selected": "prim_type", 
                              "List of All Property Use Types at Property": "all_type", 
                              "Largest Property Use Type": "lgst_type", 
                              "Largest Property Use Type - Gross Floor Area (ft²)": "lgst_type_area", 
                              "Year Built": "year_built", 
                              "Number of Buildings - Self-reported": "no_bldngs", 
                              "Occupancy": "occupancy", 
                              "Metered Areas (Energy)": "mtrd_area_energy", 
                              "ENERGY STAR Score": "energy_star", 
                              "Site EUI (kBtu/ft²)": "eui", 
                              "Weather Normalized Site EUI (kBtu/ft²)": "wthr_norm_site_eui", 
                              "Weather Normalized Site Electricity Intensity (kWh/ft²)": "wthr_norm_site_elec_int", 
                              "Weather Normalized Site Natural Gas Intensity (therms/ft²)": "wthr_norm_site_gas_int", 
                              "Weather Normalized Source EUI (kBtu/ft²)": "wthr_norm_src_eui", 
                              "Natural Gas Use (kBtu)": "gas", 
                              "Weather Normalized Site Natural Gas Use (therms)": "wthr_norm_gas", 
                              "Electricity Use - Grid Purchase (kBtu)": "elec", 
                              "Weather Normalized Site Electricity (kWh)": "wthr_norm_elec", 
                              "Total GHG Emissions (Metric Tons CO2e)": "co2_tot", 
                              "Direct GHG Emissions (Metric Tons CO2e)": "co2_dir", 
                              "Indirect GHG Emissions (Metric Tons CO2e)": "co2_ind", 
                              "Property GFA - Self-Reported (ft²)": "gfa_self", 
                              "Source EUI (kBtu/ft²)": "src_eui", 
                              "Release Date": "rel_date", 
                              "Water Required?": "water_req", 
                              "DOF Benchmarking Submission Status": "dof_bnchmrkng_sub_status"})
pp.pprint(list(df.columns))

#### Column Types

In [ ]:
# See what the column types are now
# Will want to change these based on the dimensions table above
df.info()

In [ ]:
# Set the column types appropriately
df = df.astype(dtype={"order": "int64", 
                              "id": "object", 
                              "name": "object", 
                              "prnt_id": "object", 
                              "prnt_name": "object", 
                              "brgh_blck_lt": "object", 
                              "brgh_blck_lt_self": "object", 
                              "bldng_id_no": "object", 
                              "addr_1_self": "object", 
                              "zip_code": "object", 
                              "st_no": "object", 
                              "st_name": "object", 
                              "brgh": "category", 
                              "flr_area": "float64", 
                              "prim_type": "category", 
                              "all_type": "object", 
                              "lgst_type": "category", 
                              "lgst_type_area": "float64", 
                              "year_built": "datetime64[ns]", 
                              "no_bldngs": "int64", 
                              "occupancy": "float64", 
                              "mtrd_area_energy": "category", 
                              "energy_star": "float64", 
                              "eui": "float64", 
                              "wthr_norm_site_eui": "float64", 
                              "wthr_norm_site_elec_int": "float64", 
                              "wthr_norm_site_gas_int": "float64", 
                              "wthr_norm_src_eui": "float64", 
                              "gas": "float64", 
                              "wthr_norm_gas": "float64", 
                              "elec": "float64", 
                              "wthr_norm_elec": "float64", 
                              "co2_tot": "float64", 
                              "co2_dir": "float64", 
                              "co2_ind": "float64", 
                              "gfa_self": "float64", 
                              "src_eui": "float64", 
                              "rel_date": "datetime64[ns]", 
                              "water_req": "category", 
                              "dof_bnchmrkng_sub_status": "category"})
df.info()

<a id='analyze'></a>

## Analyze

<a id='analyze-pre-processing'></a>

### Pre-processing

<a id='analyze-modeling'></a>

### Modeling

<a id='analyze-hyper-parameter-tuning'></a>

### Hyper-parameter Tuning

<a id='reflect'></a>

## Reflect